In [17]:
# 패키지 임포트

import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow import keras

print(f"version : {tf.version.VERSION}\n")

# 데이터 로드

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]
train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

# 모델 정의

def create_model():

  model = tf.keras.Sequential([
      keras.layers.Dense(512, activation='relu', input_shape=(784,)),
      keras.layers.Dropout(0.2),
      keras.layers.Dense(10)
  ])

  model.compile(
      optimizer = 'adam',
      loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
  )

  return model

model = create_model()
print("\n", model.summary(), "\n")

# 시험용 데이터셋에서 평가
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untraind model, accuracy: {:5.2f}%".format(100 * test_acc)) # 훈련되지 않은 모델은 확률 수준(~10% 정확도)에서 수행된다.


version : 2.17.1



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_22 (Dense)                     │ (None, 512)                 │         401,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 407,050 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)


 None 

32/32 - 0s - 6ms/step - loss: 2.3258 - sparse_categorical_accuracy: 0.1000
Untraind model, accuracy: 10.00%


In [22]:
# 체크포인트 콜백 사용하기

model = create_model()

checkpoint_path = "training_1/cp.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_path,
    save_weights_only = True,
    verbose = 0
)

model.fit(
    train_images,
    train_labels,
    epochs = 10,
    validation_data = (test_images, test_labels),
    callbacks = [checkpoint_callback],
    verbose = 0
)

print("\n========== 디렉토리의 하위 목록 조회 ==========\n")
print(os.listdir(checkpoint_dir))

print("\n========== 가중치 로드 전후 비교 ==========\n")
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print("[가중치 로드 전] accuracy: {:5.2f}%".format(100 * test_acc))
model = create_model()
model.load_weights(checkpoint_path)
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print("[가중치 로드 후] accuracy: {:5.2f}%".format(100 * test_acc))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



========== 디렉토리의 하위 목록 조회 ==========

['cp.weights.h5']

========== 가중치 로드 전후 비교 ==========

32/32 - 0s - 3ms/step - loss: 0.4021 - sparse_categorical_accuracy: 0.8730
[가중치 로드 전] accuracy: 87.30%


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


32/32 - 0s - 6ms/step - loss: 0.4021 - sparse_categorical_accuracy: 0.8730
[가중치 로드 후] accuracy: 87.30%


In [24]:
# 체크포인트 콜백 매개변수

checkpoint_path = "training_2/cp-{epoch:04d}.weights.h5"              # 체크포인트 경로 (epoch 값을 4자리 10진수로 출력)
checkpoint_dir = os.path.dirname(checkpoint_path)                     # 체크포인트 파일이 저장될 디렉토리 경로 추출
if not os.path.exists(checkpoint_dir): os.makedirs(checkpoint_dir)    # 디렉토리가 없으면 새로 생성
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(             # 체크포인트 생성
    filepath = checkpoint_path,                                       # 저장할 경로
    verbose = 1,                                                      # 저장 시 출력 정보 표시
    save_weights_only = True,                                         # 모델의 가중치만 저장
    save_freq = 5 * 32                                                # 배치 크기의 5배마다 저장 (즉, 5 에포크마다 모델의 가중치를 저장)
)

# `checkpoint_path` 포맷을 사용하여 가중치를 저장 (에포크 0으로 초기화)
model.save_weights(checkpoint_path.format(epoch=0))

# 새로운 콜백을 사용하여 모델을 훈련
model.fit(
    train_images,                                   # 훈련 데이터
    train_labels,                                   # 훈련 레이블
    epochs = 50,                                    # 훈련 에포크
    batch_size = 32,                                # 배치 크기
    callbacks = [checkpoint_callback],              # 콜백 추가
    validation_data = (test_images, test_labels),   # 검증 데이터 제공
    verbose = 0                                     # 훈련 중 출력 없이 진행
)

print("\n훈련 완료\n")

# 체크 포인트를 검토

    # 이진 형식의 훈련된 가중치만 포함하는 체크포인트 형식의 파일 모음에 가중치를 저장합니다. 체크포인트에는 다음이 포함됩니다.
        # - 모델의 가중치를 포함하는 하나 이상의 샤드
        # - 떤 가중치가 어떤 샤드에 저장되어 있는지 나타내는 인덱스 파일

print(f"\n========== 체크포인트 검토 ==========\n")
print(f"length:{len(os.listdir(checkpoint_dir))}")
print(os.listdir(checkpoint_dir))

# 최신 체크포인트 파일 찾기
print(f"\n========== 최신 체크포인트 파일 선택 ==========\n")
checkpoint_files = glob.glob(os.path.join(checkpoint_dir, 'cp-*.weights.h5'))
if checkpoint_files:
  latest = max(checkpoint_files, key=os.path.getctime)
  print(latest)
else:
  latest = None
  print("체크포인트 파일이 없습니다.")

# 모델 재생성 및 가중치 로드
model = create_model()
model.load_weights(latest)

# 시험셋으로 모델 평가
print("\n========== 모델 평가 ==========\n")
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * test_acc))

# 체크포인트 디렉토리 확인 및 생성
checkpoint_dir = './checkpoints'
if not os.path.exists(checkpoint_dir): os.makedirs(checkpoint_dir)



Epoch 5: saving model to training_2/cp-0005.weights.h5

Epoch 10: saving model to training_2/cp-0010.weights.h5

Epoch 15: saving model to training_2/cp-0015.weights.h5

Epoch 20: saving model to training_2/cp-0020.weights.h5

Epoch 25: saving model to training_2/cp-0025.weights.h5

Epoch 30: saving model to training_2/cp-0030.weights.h5

Epoch 35: saving model to training_2/cp-0035.weights.h5

Epoch 40: saving model to training_2/cp-0040.weights.h5

Epoch 45: saving model to training_2/cp-0045.weights.h5

Epoch 50: saving model to training_2/cp-0050.weights.h5

훈련 완료


========== 체크포인트 검토 ==========

length:12
['cp-0040.weights.h5', 'cp-0010.weights.h5', 'cp-0020.weights.h5', 'my_checkpoint.weights.h5', 'cp-0050.weights.h5', 'cp-0005.weights.h5', 'cp-0035.weights.h5', 'cp-0025.weights.h5', 'cp-0030.weights.h5', 'cp-0015.weights.h5', 'cp-0000.weights.h5', 'cp-0045.weights.h5']

========== 최신 체크포인트 파일 선택 ==========

training_2/cp-0050.weights.h5

========== 모델 평가 ==========



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


32/32 - 0s - 7ms/step - loss: 0.7766 - sparse_categorical_accuracy: 0.8760
Restored model, accuracy: 87.60%


In [25]:
# 수동으로 가중치 저장

model = create_model()
model.load_weights(latest)
model.save_weights(os.path.join(checkpoint_dir, 'my_checkpoint.weights.h5'))

# 모델 재생성 후 저장된 가중치 로드
model = create_model()
model.load_weights(os.path.join(checkpoint_dir, 'my_checkpoint.weights.h5'))

# 모델 평가
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print("accuracy: {:5.2f}%".format(100 * test_acc))

# 체크포인트 디렉토리 확인 및 생성
checkpoint_dir = './checkpoints'
if not os.path.exists(checkpoint_dir): os.makedirs(checkpoint_dir)

# 모델 저장
model.save_weights(os.path.join(checkpoint_dir, 'my_checkpoint.weights.h5'))


32/32 - 0s - 6ms/step - loss: 0.7766 - sparse_categorical_accuracy: 0.8760
accuracy: 87.60%


# Keras Model
## Keras는 아키텍처를 검사하여 다음을 모델을 저장한다.
- 가중치 값
- 모델 구조
- 모델의 훈련 구성 (.compile() 메서드에 전달하는 내용)
- 존재하는 옵티마이저와 그 상태 (훈련을 중단한 곳에서 다시 시작할 수 있게 해줌)


In [14]:
# 모델 생성

print("\n============== 생성 : 모델 1 ==============\n") # 앞서 정의한 함수 사용
model1 = create_model()
model1.fit(train_images, train_labels, epochs=5, verbose=0)
model1.save('saved_model/my_model.h5')

print("\n============== 생성 : 모델 2 ==============\n") # 저장된 모델로부터 새로운 케라스 모델을 로드
model2 = tf.keras.models.load_model('saved_model/my_model.h5')



============== 생성 : 모델 1 ==============




============== 생성 : 모델 2 ==============



In [15]:
# 모델 평가

print("\n============== 평가 : 모델 1 ==============\n")
test_loss, test_acc = model1.evaluate(test_images, test_labels, verbose=2)
print('[model1] 정확도 : {:5.2f}%'.format(100 * test_acc))
print('[model1] 모양:', model1.predict(test_images).shape)

print("\n============== 평가 : 모델 2 ==============\n")
test_loss, test_acc = model2.evaluate(test_images, test_labels, verbose=2)
print('[model2] 정확도 : {:5.2f}%'.format(100 * test_acc))
print('[model2] 모양:', model2.predict(test_images).shape)



============== 평가 : 모델 1 ==============

32/32 - 0s - 6ms/step - loss: 0.4139 - sparse_categorical_accuracy: 0.8700
[model1] 정확도 : 87.00%
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
[model1] 모양: (1000, 10)

============== 평가 : 모델 2 ==============

32/32 - 0s - 6ms/step - loss: 0.4139 - sparse_categorical_accuracy: 0.8700
[model2] 정확도 : 87.00%
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
[model2] 모양: (1000, 10)
